# Scenario Building
The purpose of this notebook is to understand the various methods that Network Wrangler can use to define and build scenarios.

The basic steps are:

1. Define base scenario (networks, projects)
2. Specify projects to add
3. Build the scenario

This notebook shows how this process can be used to build scenario upon scenario. 

In [21]:
import warnings
from pathlib import Path
import pandas as pd

%config IPCompleter.greedy=True
pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

## Define Base Scenario

A base scenario is at a minimum a `RoadwayNetwork` and `TransitNetwork`, defined with a dictionary.

In [26]:
STPAUL_DIR = Path.cwd().parent / "examples" / "stpaul"
from network_wrangler import load_roadway_from_dir, load_transit
road_net = load_roadway_from_dir(STPAUL_DIR)
transit_net = load_transit(STPAUL_DIR)
base_scenario = {
    "road_net": road_net,
    "transit_net": transit_net,
}

Skipping field outboundReferenceIds: unsupported OGR type: 5
Skipping field inboundReferenceIds: unsupported OGR type: 5


## Define "Future No Build" Scenario

In this case, we will create a future no-build scenario based on all projects in a directory with a certain filename pattern using the `glob_search` feature, but this could also be baed on a tag or whatever.

In [27]:
from network_wrangler import create_scenario
my_scenario_nobuild = create_scenario(
    base_scenario=base_scenario,
    project_card_filepath=STPAUL_DIR / "project_cards",
    filter_tags=["baseline"]
)

Base_scenario doesn't contain ['road_net', 'transit_net', 'applied_projects', 'conflicts']
Creating a deep copy of db object.            This will NOT update any referneces (e.g. from TransitNetwork)


**Check that the correct projects got added**

- These projects haven't been applied (or "built") yet, they are just specified at this point.

In [28]:
my_scenario_nobuild.queued_projects

['widen minnehaha pkwy']

## Build the "Future No Build" Scenario
 - This is where you are actually "building" the projects
 - You can check to see *all* the built projects using the `applied_projects` attribute

In [29]:
my_scenario_nobuild.apply_all_projects()

AttributeError: 'NoneType' object has no attribute 'explicit_id_selection_dict'

In [8]:
my_scenario_nobuild.applied_projects

['widen minnehaha pkwy']

## Define "Future Build" Scenario

 - Specify specific cards to add for a build scenario
 - This scenario will be tiered off of the no-build scenario and will have 'knowledge' of what projects have already been applied when checking pre-requisites.
 

 - Here, we are going to specify a list of very specific build projects.
 - In this case, we need to create `ProjectCard` instances from the YML files and then add them to the `project_card_list` keyword. 

In [10]:
BUILD_CARD_FILENAMES = [
    "road.managed_lane.simple.yml",
]

In [11]:
my_scenario_build_alt1 = Scenario.create_scenario(
    base_scenario=my_scenario_nobuild.__dict__,
    project_card_filepath=BUILD_CARD_FILENAMES,
)

my_scenario_build_alt1.applied_projects

2020-09-03 09:56:47, INFO: Creating Scenario
2020-09-03 09:56:47, INFO: Creating Scenario


['6th Street Transitway',
 'Improve Express Bus Frequency B',
 'Blue Line Service Enhancement A',
 'Blue Line Service Enhancement C',
 'Blue Line Service Enhancement B',
 'Blue Line Service Enhancement D',
 '6th St E Road Diet',
 'Improve Express Bus Frequency']

In [12]:
## Apply Build Scenario Projects

In [13]:
my_scenario_build_alt1.apply_all_projects()

2020-09-03 09:57:07, INFO: Applying test managed lane project
2020-09-03 09:57:07, INFO: Applying test managed lane project
2020-09-03 09:57:07, INFO: Applying Project to Roadway Network: test managed lane project
2020-09-03 09:57:07, INFO: Applying Project to Roadway Network: test managed lane project


In [14]:
my_scenario_build_alt1.applied_projects

['6th Street Transitway',
 'Improve Express Bus Frequency B',
 'Blue Line Service Enhancement A',
 'Blue Line Service Enhancement C',
 'Blue Line Service Enhancement B',
 'Blue Line Service Enhancement D',
 '6th St E Road Diet',
 'Improve Express Bus Frequency',
 'test managed lane project']